In [ ]:
!pip install tflite
!pip install tensorflowjs
!pip install tflite-model-maker

In [2]:
import tensorflow as tf
import tflite
import tensorflowjs
import tflite_model_maker
import pandas as pd
import tweepy, json

In [3]:
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
!nvidia-smi

Sat May 22 16:50:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
access_token="269648249-qBsHnDwZ4rXxc3MPvTfG3447vbj5w9QJwHfKkzFX"
access_token_secret="8l5MIwhpDbFWpG1aiZeJzBavf0Fhea1ynCWGdVWZmDYn2"
consumer_key="Wq00j8vB2vkPR51j5fQ4QLxUA"
consumer_secret="uZDUFvcTUZ0Lf0X8AkdzOwuBeJEZRlW93ngLz76UJgHzwTIkMW"

In [8]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [9]:
api = tweepy.API(auth)
public_tweets = api.home_timeline()

In [11]:
for tweet in public_tweets:
    #print(tweet.text)
    ...

In [23]:
class MyTweetsStreamListener(tweepy.StreamListener):
    def __init__(self, max_tweets = 100):
        # call the base class constructor
        super(MyTweetsStreamListener, self).__init__()        
        self._max_tweets = max_tweets
        self.tweets = []

    def on_status(self, status):
        tweet = status._json
        self.tweets.append(tweet)
        if len(self.tweets)  >= self._max_tweets:
            return False
        return True

    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_data disconnects the stream
            return False


In [31]:
def fetch_tweets(topics, max_tweets = 100):
    l = MyTweetsStreamListener(max_tweets)
    tweets_stream = tweepy.Stream(auth=api.auth, listener=l)
    tweets_stream.filter(languages=["en"],track=topics)
    return l.tweets


In [32]:
covid_tweets = fetch_tweets(['covid','covid19','facemask','social distancing'], max_tweets=200)

In [33]:
nasa_tweets = fetch_tweets(['Nasa','SpaceX', "Moon"], max_tweets=200)

In [80]:
def create_labeled_df(tweets_list, label):
    df = pd.DataFrame()
    for tweet in tweets_list:
        df = df.append({
            "sentence": tweet["text"],
            "label": label
            }, ignore_index=True)   
    return df

In [35]:
covid_tweets_df = create_labeled_df(covid_tweets, label="covid")
nasa_tweets_df = create_labeled_df(nasa_tweets, label="nasa")

In [36]:
pd.concat([covid_tweets_df, nasa_tweets_df]).to_csv("tweets.csv", index=None)

In [82]:
from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.text_classifier import AverageWordVecSpec
from tflite_model_maker.text_classifier import DataLoader

In [83]:
spec = AverageWordVecSpec(wordvec_dim=32)#model_spec.get('average_word_vec')

In [84]:
dataset = DataLoader.from_csv(
      filename='tweets.csv',
      text_column='sentence',
      label_column='label',
      shuffle=True,
      model_spec=spec
)

INFO:tensorflow:Saved vocabulary in /tmp/tmpwb4vntf4/3d2de8e0897d27721d0575dd86ec692c_vocab.


INFO:tensorflow:Saved vocabulary in /tmp/tmpwb4vntf4/3d2de8e0897d27721d0575dd86ec692c_vocab.


In [85]:
train_ds, val_ds = dataset.split(0.8)

In [86]:
model = text_classifier.create(train_ds, model_spec=spec, epochs=100)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Epoch 1/100
10/10 [==============================] - 1s 45ms/step - loss: 0.6935 - accuracy: 0.5122
Epoch 2/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6922 - accuracy: 0.5299
Epoch 3/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6911 - accuracy: 0.5542
Epoch 4/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6927 - accuracy: 0.5068
Epoch 5/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6920 - accuracy: 0.5476
Epoch 6/100
10/10 [==============================] - 0s 5ms/step - loss: 0.6914 - accuracy: 0.5760
Epoch 7/100
10/10 [==============================] - 0s 5ms/step - loss: 0.6905 - accuracy: 0.5956
Epoch 8/100
10/10 [==============================] - 0s 5ms/step - loss: 0.6881 - accuracy: 0.6787
Epoch 9/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6887 - accuracy: 0.6148
Epoch 10/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6860 - accuracy: 0.6610
Epoch 11

In [87]:
model.evaluate(val_ds)

3/3 [==============================] - 0s 6ms/step - loss: 0.2400 - accuracy: 0.8875


[0.24000532925128937, 0.887499988079071]

In [88]:
def predict_label(text):
    embedings = spec.preprocess(text)
    embedings_tf = tf.convert_to_tensor(embedings)    
    embedings_tf = tf.expand_dims(embedings_tf, 0)
    predictions = model.predict_top_k(embedings_tf, k =2)
    return predictions

In [89]:
predict_label("Look at the moon!, do you see that guy with mask?")

[[('nasa', 0.7493052), ('covid', 0.25069484)]]

In [91]:
predict_label("I got my first moderna shot today?")

[[('covid', 0.90607667), ('nasa', 0.09392332)]]

In [92]:
predict_label("Shoudl I be able to life in the moon?")

[[('nasa', 0.7078505), ('covid', 0.29214948)]]

In [93]:
model.export(export_dir='model',  
             export_format=[
                    ExportFormat.LABEL, 
                    ExportFormat.VOCAB, 
                    ExportFormat.TFJS, 
                    ExportFormat.TFLITE]
             )

INFO:tensorflow:Assets written to: /tmp/tmpd278c2kn/assets


INFO:tensorflow:Assets written to: /tmp/tmpd278c2kn/assets


INFO:tensorflow:Vocab file and label file are inside the TFLite model with metadata.


INFO:tensorflow:Vocab file and label file are inside the TFLite model with metadata.


INFO:tensorflow:Saved vocabulary in /tmp/tmpl2rpls2n/vocab.txt.


...

INFO:tensorflow:Assets written to: /tmp/tmpk91nif7c/assets


INFO:tensorflow:Saved vocabulary in model/vocab.txt.


INFO:tensorflow:Saved vocabulary in model/vocab.txt.


INFO:tensorflow:Saving labels in model/labels.txt


INFO:tensorflow:Saving labels in model/labels.txt


In [94]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 256, 32)           82496     
_________________________________________________________________
global_average_pooling1d_3 ( (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 66        
Total params: 83,618
Trainable params: 83,618
Non-trainable params: 0
_________________________________________________________________
